In [1]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully unins

In [2]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os
warnings.filterwarnings("ignore")

In [3]:
API_KEY = 'API_KEY HERE'

In [4]:
openai.api_key = API_KEY

In [42]:
SYSTEM_PROMPT = '''
Enhanced System Prompt for StockWiz
Role:
"I am StockWiz, your inventory forecasting assistant, specialized in predicting inventory needs by analyzing historical data, demand trends, seasonality, and other supply chain factors."

Instruction:
"I provide insights on inventory forecasting and demand prediction by examining multiple factors, such as product type, sales region, category, current stock levels, supplier lead times, and any noted weather patterns. I answer only questions relevant to inventory forecasting and demand prediction."

Constraints:

Answer questions strictly within the scope of inventory forecasting and demand prediction.
Use relevant data columns—like Region, Category, Units Sold, Stock Level, Lead Time, Supplier, and Weather Factor—when formulating specific responses.
Avoid repeating exact responses, instead tailoring answers based on available data and identified trends.
Context:
"StockWiz is designed to assist with inventory management in the Philippines. I analyze factors like demand patterns by region, seasonality impacts on specific product categories, and lead time variations from suppliers, while considering any applicable weather influences on demand."

Examples of Responses Using Additional Data Columns:

User Prompt: "What should the inventory level be for Widget A in Iloilo next month?"

Response: "Considering Widget A’s recent demand in Iloilo and the current stock level, I suggest stocking approximately X units for next month. Since the rainy season may impact demand, consider adding a buffer of about Y%."
User Prompt: "Can you forecast demand for Gadget X in Metro Manila for December?"

Response: "Gadget X typically sees a surge in Metro Manila in December. Based on last year’s data and the sales category, I recommend increasing stock by around X%. The current stock level and lead time from Supplier Z can help determine when to reorder."
User Prompt: "Should I adjust stock levels for Widget B in Cebu given the weather?"

Response: "The rainy season in Cebu often affects Widget B’s demand. Given past demand fluctuations and the weather forecast, reducing stock levels by around Y% may help manage inventory more effectively."
User Prompt: "How should I prepare stock for Tool Z in Davao with Supplier Y’s 10-day lead time?"

Response: "With Supplier Y’s lead time and Tool Z’s recent demand in Davao, maintaining at least X units as safety stock would be ideal. This will help you meet demand during potential transit delays."
User Prompt: "What reorder point should I set for Widget C in Baguio?"

Response: "For Widget C in Baguio, given a lead time of 7 days and average sales of X units, I recommend setting a reorder point of Y units to avoid stockouts, especially during high-demand periods."
This version ensures that StockWiz tailors responses based on available columns, providing more specific, actionable insights within the scope of inventory forecasting.
'''

In [32]:
prompt = "Which item has the highest demand"

## Converting the rows to embeddings and adding them to the ['combined'] column

In [34]:
df = pd.read_csv('https://raw.githubusercontent.com/asocastro/AI_First_Day_4_StockWiz/refs/heads/main/stock.csv')
df['combined'] = df.apply(lambda row : ' '.join(row.values.astype(str)), axis = 1)

docs = df['combined'].tolist()
embeddings = [get_embedding(doc, engine = "text-embedding-3-small") for doc in docs]
embedding_dim = len(embeddings[0])
embeddings_np = np.array(embeddings).astype('float32')

index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings_np)

query_embed = get_embedding(prompt, engine='text-embedding-3-small')
query_embed_np = np.array([query_embed]).astype('float32')
_, indices = index.search(query_embed_np, 2)
retrieved_docs = [docs[i] for i in indices[0]]
context = ' '.join(retrieved_docs)



struc = [{"role": "system", "content": SYSTEM_PROMPT}]
structured_prompt = f"Context:\n{context}\n\nQuery:\n{prompt}\n\nResponse:"
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struc + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struc.append({"role": "user", "content": prompt})
response = chat.choices[0].message.content

In [30]:
df.head()

,Date,Product,Category,Region,Units_Sold,Stock_Level_Before_Sale,Stock_Level_After_Sale,Sales_Revenue,Lead_Time_Days,Supplier,Weather_Factor,combined
0,2024-05-16 12:30:12.317470000,Gadget Y,Apparel,Iloilo,117,1006,889,28210.774348,7,Supplier_1,Stormy,2024-05-16 12:30:12.317470000 Gadget Y Apparel...
1,2024-05-21 03:16:21.548239230,Widget C,Household,Davao,138,541,403,50947.210664,5,Supplier_1,Cloudy,2024-05-21 03:16:21.548239230 Widget C Househo...
2,2024-05-25 18:02:30.779008461,Widget C,Apparel,Iloilo,104,267,163,18150.962212,10,Supplier_4,Cloudy,2024-05-25 18:02:30.779008461 Widget C Apparel...
3,2024-05-30 08:48:40.009777692,Gadget Y,Outdoor,Metro Manila,167,353,186,31584.148452,5,Supplier_2,Cloudy,2024-05-30 08:48:40.009777692 Gadget Y Outdoor...
4,2024-06-03 23:34:49.240546923,Widget A,Automotive,Metro Manila,33,1002,969,6139.083021,7,Supplier_3,Rainy,2024-06-03 23:34:49.240546923 Widget A Automot...


In [43]:
prompt = "predict which product will sell in iloilo"
struc = [{"role": "system", "content": SYSTEM_PROMPT}]
structured_prompt = f"Context:\n{context}\n\nQuery:\n{prompt}\n\nResponse:"
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struc + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struc.append({"role": "user", "content": prompt})
response = chat.choices[0].message.content

In [44]:
print(response)

To predict which product will sell in Iloilo, we can analyze recent sales data and demand patterns. Based on the available information, Widget A, categorized under Apparel, has shown significant sales activity in Iloilo with 127 units sold recently and a current stock level of 542 units.

Given that Widget A has a strong sales history in this region, it is likely to continue performing well. Additionally, the weather was sunny during the last recorded sales, which may positively influence demand for apparel items. 

Therefore, I recommend focusing on stocking Widget A in Iloilo, as it is likely to maintain or increase its sales momentum.
